# Tarea 2 - Modelos N-Gramas

In [1]:
import os
import re
from typing import List
from collections import Counter

import pandas as pd
import numpy as np

import nltk
from nltk import PorterStemmer, word_tokenize, sent_tokenize

from sklearn.model_selection import train_test_split

# Descargar recursos necesarios de NLTK
nltk.download('punkt')  # Tokenizador

# Configuración de NLTK
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## Generación de datos

### Funciones auxiliares

In [ ]:
def token_preprocessing(tokens: List[str], token_counts: Counter) -> List[str]:
    """
    Realiza el preprocesamiento de texto, aplicando las siguientes transformaciones:
    1. Convierte todos los tokens a minúsculas.
    3. Elimina tokens que no son palabras (puntuación).
    3. Convierte tokens de números al token NUM
    4. Aplica stemming con PorterStemmer.
    5. Reemplaza tokens con frecuencia 1 con <UNK>.

    Args:
        tokens (List[str]): Lista de tokens a preprocesar.

    Returns:
        List[str]: Lista de tokens preprocesados.
    """
    # Pasa los tokens a minúsculas
    tokens = [word.lower() for word in tokens]

    tokens = [re.sub(r'\W', '', word) for word in tokens]
    tokens = [re.sub(r'\s+[a-zA-Z]\s+', '', word) for word in tokens]
    tokens = [re.sub(r'\^[a-zA-Z]\s+', '', word) for word in tokens]

    # Sustituir números por el Token 'NUM'
    tokens = [re.sub(r'[0-9]+', 'NUM', word) for word in tokens]

    tokens = [word for word in tokens if len(word) > 0]

    # Aplica stemming
    ps = PorterStemmer()
    tokens = [ps.stem(word) for word in tokens]

    # Reemplazar tokens con frecuencia 1 con <UNK>
    tokens = [word if token_counts[word] > 1 else '<UNK>' for word in tokens]

    return tokens

In [ ]:
def text_preprocessing(text: str) -> List[str]:
    """
    Preprocesa un texto mediante la tokenización y el preprocesamiento de tokens.

    Args:
        text (str): Texto a preprocesar.

    Returns:
        List[str]: Lista de tokens preprocesados.
    """
    # Tokenizar el texto en frases
    sentences = sent_tokenize(text)

    # Tokenizar todas las frases en palabras y aplanar la lista para contar frecuencias
    all_tokens = [word.lower() for sentence in sentences for word in word_tokenize(sentence)]

    # Contar la frecuencia de los tokens en todo el texto
    token_counts = Counter(all_tokens)

    processed_sentences = []

    for sentence in sentences:
        # Añadir las etiquetas de inicio y fin de oración
        tokens = ["<s>"] + word_tokenize(sentence) + ["</s>"]

        # Preprocesar los tokens
        processed_tokens = token_preprocessing(tokens, token_counts)

        processed_tokens[0] = '<s>'
        processed_tokens[-1] = '</s>'
        processed_sentences.append(processed_tokens)

    return processed_sentences

Cargamos nuestros datasets

In [ ]:
def process_txt_files(file_path: str, csv_file_path: str, chunksize: int = 1000):
    """
    Procesa un archivo .txt línea por línea en chunks y guarda los resultados en un archivo CSV.

    Args:
        file_path (str): Ruta al archivo .txt a procesar.
        csv_file_path (str): Ruta al archivo .csv donde se guardarán los resultados.
        chunksize (int): Número de líneas a procesar en cada chunk.
    """
    chunk_list = []

    with open(file_path, 'r', encoding='utf-8') as file:
            lines = []
            for i, line in enumerate(file):
                lines.append(line.strip())
                if (i + 1) % chunksize == 0:
                    # Procesar el chunk actual
                    processed_sentences = [text_preprocessing(l) for l in lines]
                    flat_sentences = [" ".join(sentence) for sentence_chunk in processed_sentences for sentence in sentence_chunk]

                    # Crear un DataFrame con el resultado
                    chunk_df = pd.DataFrame({'processed': flat_sentences})
                    chunk_list.append(chunk_df)

                    # Guardar en CSV
                    chunk_df.to_csv(csv_file_path, mode='a', header=not pd.io.common.file_exists(csv_file_path), index=False)
                    lines = []  # Reset the list

            if lines:  # Procesar las líneas restantes si las hay
                processed_sentences = [text_preprocessing(l) for l in lines]
                flat_sentences = [" ".join(sentence) for sentence_chunk in processed_sentences for sentence in sentence_chunk]

                chunk_df = pd.DataFrame({'processed': flat_sentences})
                chunk_df.to_csv(csv_file_path, mode='a', header=not pd.io.common.file_exists(csv_file_path), index=False)

In [ ]:
def create_cvs(file_path: str, tokens: List[str]):
  with open(file_path, 'w', encoding='utf-8') as file:
    file.write('tokens\n')
    for sentence in tokens:
      file.write(' '.join(sentence) + '\n')

### Generar CSV

In [ ]:
directory = 'data/consolidated_news.txt'
process_txt_files(directory, 'data/token_news.csv')

In [ ]:
directory = 'data/consolidated_bac.txt'
process_txt_files(directory, 'data/token_bac.csv')

### Separar en train y test

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [3]:
df_twenty_news = pd.read_csv('data/token_news.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/token_news.csv'

In [ ]:
df_twenty_news.shape

In [ ]:
df_twenty_news.columns = ['text']

In [ ]:
X_train, X_test = train_test_split(df_twenty_news['text'], test_size=0.2, random_state=42)

In [ ]:
X_train.to_csv('data/20N_training', index=False, header=False)
X_test.to_csv('data/20N_testing.csv', index=False, header=False)

## Modelos

### Proceso para el dataset 20N

### Carga

In [ ]:
df_twenty_news = pd.read_csv('data/token_news.csv')

In [ ]:
df_twenty_news.shape

(471748, 1)

In [ ]:
df_twenty_news.columns = ['text']

In [ ]:
X_train, X_test = train_test_split(df_twenty_news['text'], test_size=0.2, random_state=42)

In [ ]:
X_train.to_csv('data/20N_training.csv', index=False, header=False)
X_test.to_csv('data/20N_testing.csv', index=False, header=False)

In [ ]:
df_train_twenty_news = pd.read_csv('data/20N_training.csv')
df_train_twenty_news.shape

(377397, 1)

In [ ]:
df_test_twenty_news = pd.read_csv('data/20N_testing.csv')
df_test_twenty_news.shape

(94349, 1)

### N-Gramas

#### Funciones auxiliares

In [4]:
def custom_tokenizer(text):
    """
    Tokeniza un texto personalizado (para los casos de <s>, </s> y <UNK>).

    Args:
    text: El texto a tokenizar.

    Returns:
    Una lista de tokens.
    """
    tokens = re.findall(r"<[^>]+>|[\w']+|[.,!?;]", text)
    return tokens

Cálculo de probabilidades para **unigramas**, **bigramas** y **trigramas**.

In [5]:
def calculate_unigrams_probabilities(corpus: List[str]) -> dict:
  """
  Calcula las probabilidades de unigramas.

  Args:
    corpus: Lista de palabras.

  Returns:
    dict: Un diccionario con las probabilidades de unigramas.
  """
  unigrams = Counter(corpus.split())
  V = len(unigrams) # tamaño del vocabulario
  total_words_in_corpus = sum(unigrams.values())

  # Se aplica el suavizado de Laplace
  probabilities = {}
  for word, count in unigrams.items():
    probabilities[word] = (count + 1) / (total_words_in_corpus + V)

  return probabilities

In [6]:
def calculate_bigrams_probabilities(corpus: List[str]) -> dict:
  """
  Calcula las probabilidades de bigramas.

  Args:
    corpus: Lista de palabras.

  Returns:
    dict: Un diccionario con las probabilidades de bigramas.
  """
  unigrams_count = Counter(corpus.split())

  bigram_tokens = custom_tokenizer(corpus)
  bigrams_list = list(nltk.bigrams(bigram_tokens))
  bigrams = Counter(bigrams_list)
  V = len(bigrams) # tamaño del vocabulario

  # Se aplica el suavizado de Laplace
  probabilities = {}
  for bigram, count in bigrams.items():
    probabilities[bigram] = (count + 1) / (unigrams_count[bigram[0]] + V) # [0] es el primer token del bigrama

  return probabilities

In [7]:
def calculate_trigrams_probabilities(corpus: List[str]) -> dict:
  """
  Calcula las probabilidades de bigramas.

  Args:
    corpus: Lista de palabras.

  Returns:
    dict: Un diccionario con las probabilidades de bigramas.
  """
  tokens = custom_tokenizer(corpus)

  bigrams_list = list(nltk.bigrams(tokens))
  bigrams_count = Counter(bigrams_list)

  trigrams_list = list(nltk.trigrams(tokens))
  trigrams = Counter(trigrams_list)
  V = len(trigrams) # tamaño del vocabulario

  # Se aplica el suavizado de Laplace
  probabilities = {}
  for trigram, count in trigrams.items():
    probabilities[trigram] = (count + 1) / (bigrams_count[(trigram[:2])] + V) # [:2] es el bigrama de w_i-2 y w_i-1

  return probabilities

#### Probabilidades de cada modelo

In [8]:
with open('data/20N_training.csv', 'r') as f:
  docs = f.read()

In [9]:
unigrams_probabilities = calculate_unigrams_probabilities(docs)
for trigram, prob in list(unigrams_probabilities.items())[:5]:
    print(f"{trigram}: {prob}")

<s>: 0.046724057927090466
<UNK>: 0.2907921667788344
from: 0.003012186914554917
</s>: 0.046724057927090466
i: 0.013219204505330321


In [10]:
bigrams_probabilities = calculate_bigrams_probabilities(docs)
for trigram, prob in list(bigrams_probabilities.items())[:5]:
    print(f"{trigram}: {prob}")

('<s>', '<UNK>'): 0.1218935563803793
('<UNK>', 'from'): 0.003503323336405251
('from', '<UNK>'): 0.012334273897414092
('<UNK>', '</s>'): 0.06298874017241446
('</s>', '<s>'): 0.43354712121577643


In [11]:
trigrams_probabilities = calculate_trigrams_probabilities(docs)
for trigram, prob in list(trigrams_probabilities.items())[:5]:
    print(f"{trigram}: {prob}")

('<s>', '<UNK>', 'from'): 0.00015126854505457393
('<UNK>', 'from', '<UNK>'): 0.0021050996473895955
('from', '<UNK>', '</s>'): 0.0003244385687090521
('<UNK>', '</s>', '<s>'): 0.10065140077875707
('</s>', '<s>', 'i'): 0.015974772202682324


#### Generar archivos con probabilidades

In [12]:
unigrams_df = pd.DataFrame(unigrams_probabilities.items(), columns=['N-Gram', 'Probability'])
bigrams_df = pd.DataFrame(bigrams_probabilities.items(), columns=['N-Gram', 'Probability'])
trigrams_df = pd.DataFrame(trigrams_probabilities.items(), columns=['N-Gram', 'Probability'])

In [13]:
unigrams_df.to_csv('data/20N_unigrams.csv', index=False)
bigrams_df.to_csv('data/20N_bigrams.csv', index=False)
trigrams_df.to_csv('data/20N_trigrams.csv', index=False)

#### Generación de texto